In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [52]:
mnist.train.labels=0
print mnist.train.labels.shape

AttributeError: can't set attribute

In [3]:
import tensorflow as tf
from tensorflow.contrib import learn
import numpy as np
import random
sess = tf.InteractiveSession()

In [4]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [41]:
## shuffle 10 classes
shuffle_pre = 0.00;
tot_len = len(mnist.train.labels)
change_len = int(tot_len*shuffle_pre)
indexlist = random.sample(range(tot_len),change_len)
for curp in indexlist:
	tind=-1
	for j, xx in enumerate(mnist.train.labels[curp]):
		if xx==1:
			tind = j
			break
	mnist.train.labels[curp][tind]=int(0)
	tind+=random.randint(1,9)
	tind%=10
	mnist.train.labels[curp][tind]=int(1)

In [5]:
## shuffle 2 classes
shuffle_pre = 0.50;
tot_len = len(mnist.train.labels)
change_len = int(tot_len*shuffle_pre)
indexlist = random.sample(range(tot_len),change_len)
for curp in indexlist:
	tind=-1
	for j, xx in enumerate(mnist.train.labels[curp]):
		if xx==1:
			tind = j
			break
	mnist.train.labels[curp][tind]=int(0)
	if tind==4:
		tind = 6
	if tind==6:
		tind = 4
	mnist.train.labels[curp][tind]=int(1)

In [6]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

#W = tf.Variable(tf.zeros([784,10]))
#b = tf.Variable(tf.zeros([10]))

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


In [7]:
open("accuracy0.3", 'w').close()

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(200000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  if i%1000 == 0:
    test_accuracy = accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})
    with open("accuracy0.3","a") as text_file:
        text_file.write("step %d, test accuracy %g\n"%(i,test_accuracy))
    print("test accuracy %g"%accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


step 0, training accuracy 0.1
test accuracy 0.0934
step 100, training accuracy 0.76
step 200, training accuracy 0.82
step 300, training accuracy 0.86
step 400, training accuracy 0.9
step 500, training accuracy 0.78
step 600, training accuracy 0.92
step 700, training accuracy 0.92
step 800, training accuracy 0.86
step 900, training accuracy 0.94
step 1000, training accuracy 0.94


KeyboardInterrupt: 

In [4]:
#sess.run(tf.global_variables_initializer())


In [5]:
#y = tf.matmul(x,W) + b
#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
#for _ in range(1000):
#  batch = mnist.train.next_batch(100)
#  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [6]:
#correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9196


In [4]:
shuffle_pre = 0.90;
tot_len = len(mnist.train.labels)
change_len = int(tot_len*shuffle_pre)
curp = 0
max_step = int(tot_len/change_len)
for i in range(change_len):
	curp+=random.randint(1,max_step)
	tind=-1
	for j, xx in enumerate(mnist.train.labels[curp]):
		if xx==1:
			tind = j
			break
	mnist.train.labels[curp][tind]=int(0)
	tind+=random.randint(1,9)
	tind%=10
	mnist.train.labels[curp][tind]=int(1)